# Creating a Model: Mensa

Source/Basis: https://www.gurobi.com/documentation/8.1/examples/

## Model Description

As you might already know, our new mensa in Garching is now based on a self-service system. A side-effect of this is, that one can choose exactly how large their portion should be. Besides having many advantages, this system also makes everyone responsible for his own healthy diet. The below table depicts the minimum and maximum recommended intake for different nutritional values.

| Nutritional Value | Min intake | Max intake |
|-------------------|------------|------------|
| calories          | 1800       | 2200       |
| protein           | 91         | $\infty$   |
| fat               | 0          | 65         |
| sugars            | 0          | 40         |

The individual dishes from which it is possible to choose all come with a price and their respective nutritional values:

| Dish      | Price /100g | calories /100g | protein /100g | fat /100g | sugars /100g |
|-----------|-------------|----------------|---------------|-----------|--------------|
| chicken   | 1.50        | 239            | 27            | 1.1       | 0            |
| fries     | 0.75        | 312            | 3.4           | 15        | 0.3          |
| macaroni  | 0.75        | 164            | 7             | 10        | 1.6          |
| pizza     | 1.00        | 266            | 11            | 10        | 3.6          |
| salad     | 0.50        | 152            | 1             | 5         | 12           |
| milk      | 0.33        | 149            | 3.4           | 1         | 5            |
| ice cream | 1.00        | 207            | 2             | 7         | 21           |

Your objective is to find a meal arrangement, that complies with the nutritional recommendations while being as cheap as possible.

### Step 1: Loading Gurobi

In [1]:
from gurobipy import *

### Step 2: Create a Model

In [2]:
m = Model()

Academic license - for non-commercial use only


### Step 3: Create Variables

Above tables are already given to you as dictionaries. Recall, that variables can be created from lists and the keys of dictionaries.

In [3]:
dishes = {'chicken':{'price':1.5, 'calories':239, 'protein':27, 'fat':1.1, 'sugars':0},
         'fries':{'price':0.75, 'calories':312, 'protein':3.4, 'fat':15, 'sugars':0.3},
         'macaroni':{'price':0.75, 'calories':164, 'protein':7, 'fat':10, 'sugars':1.6},
         'pizza':{'price':1.0, 'calories':266, 'protein':11, 'fat':10, 'sugars':3.6},
         'salad':{'price':0.5, 'calories':152, 'protein':1, 'fat':5, 'sugars':12},
         'milk':{'price':0.33, 'calories':149, 'protein':3.4, 'fat':1, 'sugars':5},
         'ice cream':{'price':1.0, 'calories':207, 'protein':2, 'fat':7, 'sugars':21},
         }
nutrition = {'calories':{'min':1800,'max':2200},
            'protein':{'min':91,'max':GRB.INFINITY},
            'fat':{'min':0,'max':65},
            'sugars':{'min':0,'max':40},
            }

In [4]:
x = m.addVars(dishes,lb=0,vtype=GRB.CONTINUOUS)

### Step 4: Add Objective

Recall, that you can set the objective on the model or on each variable.

In [5]:
for d,dval in dishes.items():
    x[d].obj=-dval['price']
    
m.ModelSense=GRB.MAXIMIZE

### Step 5: Add Constraints

Recall the different options to create constraints and how to work with dictionaries.

In [6]:
for n,nval in nutrition.items():
    expr = 0
    for d,dval in dishes.items():
        expr+=dval[n]*x[d]
    m.addConstr(expr>=nval['min'])
    m.addConstr(expr<=nval['max'])

### Step 6: Solve

In [7]:
m.optimize()


Optimize a model with 8 rows, 7 columns and 54 nonzeros
Coefficient statistics:
  Matrix range     [3e-01, 3e+02]
  Objective range  [3e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 2e+03]
Presolve removed 4 rows and 0 columns
Presolve time: 0.01s
Presolved: 4 rows, 8 columns, 28 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   1.472500e+02   0.000000e+00      0s
       3   -6.2714370e+00   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds
Optimal objective -6.271436986e+00


### Step 7: Display Solution (in a meaningful way)

In [8]:
for d in dishes:
    print(f"{d}: {x[d].x*100}g")

chicken: 234.4330115911304g
fries: 15.741141126245072g
macaroni: 0.0g
pizza: 0.0g
salad: 0.0g
milk: 799.0555315324252g
ice cream: 0.0g


### Step 8: Interpret Outcome and Refinement of Model

If you worked correctly, then you should have recieved an optimal mealplan. Probably, the amount of milk is on the heavy side though. In order to get a really reasonable result, you should limit the amount of milk to `200g`. Also, the diet does not include any vitamines. Please add at least `100g` of salad to your meal choice.

Add the constraints to the existing model and optimize again to see the resulting meal plan.

In [9]:
m.addConstr(x['milk']<=2)
m.addConstr(x['salad']>=1)
m.optimize()
for d in dishes:
    print(f"{d}: {x[d].x*100}g")

Optimize a model with 10 rows, 7 columns and 56 nonzeros
Coefficient statistics:
  Matrix range     [3e-01, 3e+02]
  Objective range  [3e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+03]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -6.2714370e+00   1.118489e+02   0.000000e+00      0s
       2   -7.0034059e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective -7.003405944e+00
chicken: 280.7420448274956g
fries: 217.63670284047615g
macaroni: 0.0g
pizza: 0.0g
salad: 100.0g
milk: 200.0g
ice cream: 0.0g
